questa prima parte va ad importare tutto quello che mi servirà dopo e fa partire Chrome

In [1]:
from selenium import webdriver                         # import the webdriver API 
from selenium.webdriver.common.keys import Keys        # import the keys of the keyboard such as RETURNS, F1, ALT, ...
import re                                              # import regular expression

from selenium.common.exceptions import NoSuchElementException #importo l'eccezione, in modo che la possa riconoscere
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from selenium.webdriver.chrome.options import Options
chromeOptions = Options()  
#chromeOptions.add_argument("--headless") 
#driver = webdriver.Chrome(options=chromeOptions) # lancio il driver in modalità Headless

driver = webdriver.Chrome()                            # run the browser
                                           

Adesso vado a recuperare i links salvati nel file "dataLinks.json" e li stampo a schermo

In [2]:
import json

links = []    # dichiaro l'array links

with open('dataLinks.json') as json_file:
    dataLinks = json.load(json_file)
    print(dataLinks)
    
    print("***************************")

    for link in dataLinks:
        print(link["link"])
        
        links.append(link["link"])    # metto i links recuperati da "dataLinks", in "links"  


[{'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAS'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAT'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAR'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_POT'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTP'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTM'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_BAL'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_NIT'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_FIT'}]
***************************
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAS
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAT
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAR
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_POT
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTP
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTM
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_BAL
http:/

questa parte seleziona i link di mio interesse ed apre tutti i tab contenenti quelle pagine

In [3]:
# parameters and output data
#    url         = links[0]  # link alla pagina
id_n_pulsante_m = "apexir_ACTIONSMENUROOT"  # id del pulsante menu a tendina 
id_n_menu_tenina = "apexir_ACTIONSMENU"     # id menu a tendina 
        
#    driver.get(url)  # get the web page
select = re.compile("(\w*MAS$)|(\w*MAT$)|(\w*POT$)")    #espressione regolare per selezionare i link di mio interesse
i = 0    #la prima tab
    
for link in links:
    if (select.search(links[i])):# usa select come valore di controllo, vede se corrisponde ad una delle opzioni
        driver.execute_script("window.open('about:blank', 'tab"+str(i)+"');")
        driver.switch_to.window("tab"+str(i))
        driver.get(links[i])
        
    i+=1   # serve a dirgli il numero di tab da usare  


la parte sotto trova i filtri e deve toglierli

In [4]:
# inizio parte che individua e toglie i filtri *****************************************************************
temp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")
len(temp)  



1

questa parte sotto ha due except in quanto nella versione precedente di chromeDriver dava problemi con i tempi di attesa 

In [5]:
    tmp = temp
    try:
        while (len(tmp) != 0):#tmp != []
            driver.find_element_by_css_selector("img[title='Rimuovi filtro']").click()
            tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")
    except:
        try:
            while (len(tmp) != 0):#tmp != []
                driver.find_element_by_css_selector("img[title='Rimuovi filtro']").click()
                tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")            
        except:
            print("An exception occurred")

# fine parte che gestisce i filtri *******************************************************************************

An exception occurred


 Questa parte qui sotto servirebbe a scaricare il csv ma a causa di un problema riscontrato con il server della regione, riuslta impossibile scaricare il csv tramite programma.
 

In [6]:
def get_CSV():
   
 #   id_csv = "apexir_dl_CSV"                    # individua l'id del csv 
#    driver.find_element_by_id(id_csv).click()   # scarica il csv
    
    print("Ho scaricqato il file :)")            # li faccio stampare questo come variante del file che non può scaricaere 
                                                 # la causa è il problema che le dicevo nella chiamata 
    #return 1


Questo serve a scaricare la pagina usando wget

si rende necessario in quanto le coordinate geografiche non hanno un link scaricabile, ma un link alla pagina con i dati

In [7]:
def coord():
        #****************************** nella pagina dove sono, cerca il link con quel nome
        #questo è inserito in ctrl_t() in modo che quando fa i vari controlli per scaricare, controlla anche questo
        #inoltre per semplicità è inserito come ultimo comando, cioé
        #fa tutto il resto, poi questo, e poi cambia tab 
        
    import wget 
    
    driver.find_element_by_link_text('GEOJSON').click()
    #url = driver.page_source
    url = driver.current_url
    print(url)
    wget.download(url)#,out="") # download
        
        # la versione sotto funziona ma scarica tutto, compresi i tag html che mi danno fastidio
        #with open('page.geojson', 'w') as f:
        #    f.write(driver.page_source)
        
        #******************************

Parte che serve ad interagire (aprire e selezionare) le parti nella tendina per scaricare il csv. E' anche presente la parte che dovrebbe scaricare effettivamente il file get_CSV()

In [8]:

def ctrl_t():

    try:

        elem = driver.find_element_by_id(id_n_pulsante_m)
        
        # inizio parte che trova ed apre il menu a tendina 
        driver.find_element_by_id(id_n_pulsante_m).click()     # seleziono il pulsante menu a tendina
    
        tendina = driver.find_element_by_id(id_n_menu_tenina)  # individua il menu a tendina
        tendina.find_element_by_css_selector("a[title='Scarica']").click()  # fa click su scarica
        # a questo punto il dawnload deve ancora partire, lo fa la funzione "get_CSV()" 
        
        get_CSV()
        # fine parte che apre il menu a tendina

        print("sono arrivato in fondo al primo caso")
    
    except NoSuchElementException:   # eccezione in quanto è una sola la pagina di questo tipo, diversa dalle altre
        #controlla e scarica i due file da link
        links_div = driver.find_elements_by_class_name("t20CVS")    # individua i link e li mette in links_div
        for i in links_div:
            print(i)
            i.find_element_by_tag_name('a').click()    # fa click sui link per scaricarli
        coord()
        print("fine della eccezione")

In [9]:
#va messo in un for che mi cicla tutti i tab
#ctrl_t()

anche qui forse ci vorrebbe un piccolo tempo di attesa 

verificare se eseguendolo headless non è necessario

In [10]:
#driver.switch_to.window(driver.window_handles[1]) #mi fa muovere tra i tab
# len(driver.window_handles) # mi da il numero di tab aperti nella pagina corrente
for i in range(1,len(driver.window_handles)):
    driver.switch_to.window(driver.window_handles[i])
    #################################
    tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")
    try:
        while (len(tmp) != 0):#tmp != []
            driver.find_element_by_css_selector("img[title='Rimuovi filtro']").click()
            tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")
    except:
        try:
            while (len(tmp) != 0):#tmp != []
                driver.find_element_by_css_selector("img[title='Rimuovi filtro']").click()
                tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")            
        except:
            print("An exception occurred")
    ################################
    ctrl_t()
    driver.implicitly_wait(2)
    #get_CSV()

print("finito")

An exception occurred
Ho scaricqato il file :)
sono arrivato in fondo al primo caso
<selenium.webdriver.remote.webelement.WebElement (session="8b3739b5b79fe04534980c23b02747e6", element="0f3775b5-3cdd-4099-9bb6-291256a1a255")>
<selenium.webdriver.remote.webelement.WebElement (session="8b3739b5b79fe04534980c23b02747e6", element="b4bd1333-c343-4b38-9f18-b8bdbf99e546")>
http://sira.arpat.toscana.it/sira/Acque/GEOJSON/MAT_STAZIONI.geojson
fine della eccezione
Ho scaricqato il file :)
sono arrivato in fondo al primo caso
finito


In [ ]:
#get_CSV()
#driver.implicitly_wait(600)

In [ ]:
driver.quit()